In [70]:
import pygame
import os
import time
import random
from tkinter import *
from tkinter import filedialog, messagebox
from mutagen.mp3 import MP3


# setting

In [71]:
pygame.mixer.init()
def music():
    class MusicPlayer:
        def __init__(self, root):
            self.root = root
            self.root.title("🎵 موزیک پلیر حرفه‌ای 🎵")
            self.root.geometry("700x550")
            self.root.configure(bg="#222")
            self.root.attributes("-topmost", True)

            # Playlist و تنظیمات
            self.playlist = []
            self.current_index = 0
            self.paused = False
            self.song_length = 0
            self.start_time = 0
            self.offset = 0
            self.shuffle = False
            self.repeat = "None"  # None, One, All

            # Label آهنگ
            self.label = Label(root, text="هیچ آهنگی انتخاب نشده", bg="#222", fg="white", font=("Arial", 14))
            self.label.pack(pady=10)

            # Frame لیست آهنگ + Scrollbar
            list_frame = Frame(root, bg="#222")
            list_frame.pack(pady=5)

            self.listbox = Listbox(list_frame, bg="#333", fg="white", width=60, height=12, selectbackground="#555", font=("Arial",12))
            self.listbox.pack(side=LEFT, fill=BOTH, expand=True)
            self.listbox.bind('<Double-Button-1>', self.play_selected)

            self.scrollbar = Scrollbar(list_frame)
            self.scrollbar.pack(side=RIGHT, fill=Y)
            self.listbox.config(yscrollcommand=self.scrollbar.set)
            self.scrollbar.config(command=self.listbox.yview)

            # دکمه‌ها
            control_frame = Frame(root, bg="#222")
            control_frame.pack(pady=5)

            Button(control_frame, text="📂 افزودن آهنگ", command=self.add_songs, bg="#4CAF50", fg="white", font=("Arial",12)).grid(row=0,column=0,padx=5)
            Button(control_frame, text="◀ قبلی", command=self.prev_song, bg="#2196F3", fg="white", font=("Arial",12)).grid(row=0,column=1,padx=5)
            Button(control_frame, text="▶ بعدی", command=self.next_song, bg="#2196F3", fg="white", font=("Arial",12)).grid(row=0,column=2,padx=5)
            Button(control_frame, text="⏸ توقف/ادامه", command=self.pause_resume, bg="#f44336", fg="white", font=("Arial",12)).grid(row=0,column=3,padx=5)
            Button(control_frame, text="⏹ توقف", command=self.stop_song, bg="#9C27B0", fg="white", font=("Arial",12)).grid(row=0,column=4,padx=5)

            # نوار صدا
            self.volume_scale = Scale(root, from_=0, to=100, orient="horizontal", label="🔊 صدا",
                                    command=self.set_volume, bg="#222", fg="white", length=600)
            self.volume_scale.set(70)
            self.volume_scale.pack(pady=5)
            pygame.mixer.music.set_volume(0.7)

            # نوار پیشرفت
            self.progress = Scale(root, from_=0, to=100, orient="horizontal", length=600)
            self.progress.pack(pady=5)
            self.progress.bind("<ButtonRelease-1>", self.seek_release)

            # زمان
            self.time_label = Label(root, text="00:00 / 00:00", bg="#222", fg="white", font=("Arial",12))
            self.time_label.pack(pady=5)

            # Shuffle و Repeat
            options_frame = Frame(root, bg="#222")
            options_frame.pack(pady=5)
            self.shuffle_btn = Button(options_frame, text="Shuffle Off", command=self.toggle_shuffle, bg="#555", fg="white")
            self.shuffle_btn.grid(row=0,column=0,padx=5)
            self.repeat_btn = Button(options_frame, text="Repeat: None", command=self.toggle_repeat, bg="#555", fg="white")
            self.repeat_btn.grid(row=0,column=1,padx=5)

            # بروزرسانی نوار پیشرفت
            self.update_progress()

            # بستن پنجره
            self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        # افزودن آهنگ‌ها به Playlist
        def add_songs(self):
            files = filedialog.askopenfilenames(title="انتخاب آهنگ", filetypes=[("Audio Files","*.mp3 *.wav")])
            if files:
                for f in files:
                    self.playlist.append(f)
                    self.listbox.insert(END, os.path.basename(f))
                if len(files) > 0 and not pygame.mixer.music.get_busy():
                    self.current_index = 0
                    self.load_song(self.playlist[self.current_index])

        # بارگذاری آهنگ
        def load_song(self, file):
            self.current_file = file
            self.label.config(text=f"در حال پخش: {os.path.basename(file)}")
            try:
                if file.lower().endswith(".mp3"):
                    audio = MP3(file)
                    self.song_length = int(audio.info.length)
                else:
                    snd = pygame.mixer.Sound(file)
                    self.song_length = int(snd.get_length())
                self.progress.config(to=self.song_length)
                self.offset = 0
                pygame.mixer.music.load(file)
                pygame.mixer.music.play()
                self.start_time = time.time()
                self.paused = False

                # انتخاب آهنگ در لیست
                self.listbox.selection_clear(0, END)
                self.listbox.selection_set(self.current_index)
                self.listbox.activate(self.current_index)
            except Exception as e:
                messagebox.showerror("خطا", f"مشکل در پخش: {e}")

        # دابل کلیک روی آهنگ
        def play_selected(self, event):
            selection = self.listbox.curselection()
            if selection:
                self.current_index = selection[0]
                self.load_song(self.playlist[self.current_index])

        # Play/Pause
        def pause_resume(self):
            if not self.playlist: return
            if not self.paused:
                pygame.mixer.music.pause()
                self.paused = True
                self.offset = time.time() - self.start_time
            else:
                pygame.mixer.music.unpause()
                self.paused = False
                self.start_time = time.time() - self.offset

        # توقف
        def stop_song(self):
            if not self.playlist: return
            pygame.mixer.music.stop()
            self.paused = False
            self.offset = 0
            self.progress.set(0)

        # آهنگ بعدی
        def next_song(self):
            if not self.playlist: return
            if self.shuffle:
                self.current_index = random.randint(0, len(self.playlist)-1)
            else:
                self.current_index +=1
                if self.current_index >= len(self.playlist):
                    if self.repeat=="All":
                        self.current_index=0
                    else:
                        self.current_index = len(self.playlist)-1
            self.load_song(self.playlist[self.current_index])

        # آهنگ قبلی
        def prev_song(self):
            if not self.playlist: return
            if self.shuffle:
                self.current_index = random.randint(0, len(self.playlist)-1)
            else:
                self.current_index -=1
                if self.current_index<0:
                    if self.repeat=="All":
                        self.current_index=len(self.playlist)-1
                    else:
                        self.current_index=0
            self.load_song(self.playlist[self.current_index])

        # صدا
        def set_volume(self,val):
            pygame.mixer.music.set_volume(int(val)/100)

        # نوار پیشرفت
        def update_progress(self):
            if self.playlist and (pygame.mixer.music.get_busy() or self.paused):
                if self.paused:
                    pos = int(self.offset)
                else:
                    pos = int(time.time()-self.start_time)
                    if pos>self.song_length:
                        # پایان آهنگ
                        if self.repeat=="One":
                            self.load_song(self.current_file)
                            pos=0
                        else:
                            self.next_song()
                            pos=0
                self.progress.set(pos)
                elapsed = time.strftime("%M:%S", time.gmtime(pos))
                total = time.strftime("%M:%S", time.gmtime(self.song_length))
                self.time_label.config(text=f"{elapsed} / {total}")
            self.root.after(500,self.update_progress)

        def seek_release(self,event):
            if self.playlist:
                pos=int(self.progress.get())
                self.offset=pos
                pygame.mixer.music.stop()
                pygame.mixer.music.play(start=self.offset)
                self.start_time=time.time()-self.offset
                self.paused=False

        # Shuffle
        def toggle_shuffle(self):
            self.shuffle=not self.shuffle
            self.shuffle_btn.config(text=f"Shuffle {'On' if self.shuffle else 'Off'}")

        # Repeat
        def toggle_repeat(self):
            if self.repeat=="None":
                self.repeat="One"
            elif self.repeat=="One":
                self.repeat="All"
            else:
                self.repeat="None"
            self.repeat_btn.config(text=f"Repeat: {self.repeat}")

        # بستن پنجره
        def on_close(self):
            pygame.mixer.music.stop()
            self.root.destroy()


    if __name__=="__main__":
        root = Tk()
        app = MusicPlayer(root)
        root.mainloop()

# loop

In [72]:
loop = Tk()
loop.configure(bg="#222")
loop.title('Clock Practice')
loop.geometry('200x100')
loop.attributes("-topmost", True)

# دکمه‌ها
setting1 = Button(loop, text="music", command=music)
setting1.place(rely=0.85, relx=0.25, anchor="center")

setting2 = Button(loop, text="setting")
setting2.place(rely=0.85, relx=0.75, anchor="center")

# ساعت
lable1 = Label(loop, text='hello world!', font=('Arial', 24), fg='green', bg="#222")
lable1.pack()
lable2 = Label(loop, text='hello world!', font=('Arial', 12), fg='white', bg="#222")
lable2.pack()

def mytime():
    hour = time.strftime('%I')
    minute = time.strftime("%M")
    second = time.strftime("%S")
    am_pm = time.strftime("%p")
    day = time.strftime("%A")
    zone = time.strftime("%Z")
    mytext = hour + ':' + minute + ':'+ second+' ' + am_pm
    mytext2 = day + "   " + zone
    lable1.config(text=mytext)
    lable2.config(text=mytext2)
    lable1.after(1000, mytime)

mytime()
loop.mainloop()

# time